# Importing Required Libraries

In [6]:
import os
import logging
import time
from datetime import datetime
import mysql.connector
from mysql.connector import Error
from selenium import webdriver
from sqlalchemy import create_engine
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import streamlit as st

# Setting Up Logging and Chrome Profile Path

In [ ]:
# Get the user's home directory
home_dir = os.path.expanduser("~")

# Set the log file path to a subdirectory in the user's home directory with a timestamp
log_file_path = os.path.join(home_dir, f"scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s %(message)s')

chrome_profile_path = 'C:/Users/suffu/AppData/Local/Google/Chrome/User Data/Default'

# Web Scraping and Data Extraction with Selenium 

In [39]:
def scrape_route(route_link):
    print(route_link)
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-software-rasterizer')
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
    options.add_argument(f"user-data-dir={chrome_profile_path}") 

    # Provide the path to your chromedriver executable
    chromedriver_path = Service(executable_path="C:/Project/chromedriver-win64/chromedriver-win64/chromedriver.exe")

    driver = webdriver.Chrome(service=chromedriver_path, options=options)
    driver.set_page_load_timeout(60)

    try:
        driver.get(route_link)
        time.sleep(10)
        scroll_pause_time = 4
        screen_height = driver.execute_script("return window.screen.height")
        i = 1
        while True:
            driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
            i += 1
            time.sleep(scroll_pause_time)
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            if screen_height * i > scroll_height:
                break

        # Wait for the page to load
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".clearfix")))

        try:
            view_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".button"))
            )
            driver.execute_script("arguments[0].click();", view_button)
            time.sleep(5)

        except Exception as e:
            logging.error(f"Error for clicking button data: {e}")
                
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".row-sec.clearfix")))
        time.sleep(10)

        # Extract data from the page
        #bus_data = []

        route_name = driver.find_element(By.CSS_SELECTOR, ".D136_h1").text.strip()
        print(f"Route Name: {route_name}")
        
        print(route_link)
        
        bus_data = []
        bus_cards = driver.find_elements(By.CSS_SELECTOR, ".row-sec.clearfix")
        time.sleep(5)

        if not bus_cards:
            logging.warning("No bus cards found.")
            print("No bus cards found.")

        for bus in bus_cards:
            try:
                busname = bus.find_element(By.CSS_SELECTOR, ".travels").text.strip()
                print(busname)
                print(busname.startswith('Ad'))
                if busname.startswith('Ad'):
                    busname.replace('Ad',' ').strip()
                    print(busname)

                bustype = bus.find_element(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus").text.strip()
                print(bustype)

                departing_time = bus.find_element(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold").text.strip()
                print(departing_time)

                duration = bus.find_element(By.CSS_SELECTOR, ".dur.l-color.lh-24").text.strip()
                print(duration)

                reaching_time = bus.find_element(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline").text.strip()
                print(reaching_time)

                rating_element = bus.find_element(By.CSS_SELECTOR, ".row-sec.clearfix .lh-18.rating")        
                # Check the rating class and extract the star rating
                if "rat-green" in rating_element.get_attribute("class"):
                    star_rating = rating_element.find_element(By.CSS_SELECTOR, "span").text.strip()
                elif "rat-red" in rating_element.get_attribute("class"):
                    star_rating = rating_element.find_element(By.CSS_SELECTOR, "span").text.strip()
                elif "rat-orange" in rating_element.get_attribute("class"):
                    star_rating = rating_element.find_element(By.CSS_SELECTOR, "span").text.strip()
                else:
                    star_rating = "NA"
                print(star_rating)

                price = bus.find_element(By.CSS_SELECTOR, ".fare.d-block").text.strip()
                print(price)
                if price is None:
                    price = "NA"
                else:
                    price = price.replace('₹', '').replace(',', '').replace('Rs', ' ').strip()

                seats_available = bus.find_element(By.CSS_SELECTOR, ".seat-left.m-top-30").text.split()[0].strip()
                print(seats_available)

                bus_data.append({
                    'route_name': route_name,
                    'route_link': route_link,
                    'busname': busname if busname else "NA",
                    'bustype': bustype if bustype else "NA",
                    'departing_time': departing_time,
                    'duration': duration if duration else "NA",
                    'reaching_time': reaching_time,
                    'star_rating': star_rating if star_rating else "NA",
                    'price': price if price else "NA",
                    'seats_available': seats_available if seats_available else "NA"
                })

                logging.info(f"Scraped data: {bus_data[-1]}")
            except Exception as e:
                logging.error(f"Error extracting data for a bus card: {e}")
        
        if not bus_data:
            logging.warning(f"No data scraped for {route_link}")

        return bus_data
    except Exception as e:
        logging.error(f"Error scraping route {route_link}: {e}")
        return []
    finally:
        driver.quit()

# List of route links to scrape (Add more as needed)
route_links = [
        'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore?fromCityId=74661&toCityId=122&fromCityName=Kozhikode&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore?fromCityId=653&toCityId=122&fromCityName=Chittoor%20(Andhra%20Pradesh)&toCityName=Bangalore&busType=Any&opId=0&onward=19-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/kadapa-to-hyderabad?fromCityId=284&toCityId=124&fromCityName=Kadapa&toCityName=Hyderabad&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/mysore-to-kozhikode?fromCityId=129&toCityId=74661&fromCityName=Mysore&toCityName=Kozhikode&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/kolkata-to-suri?fromCityId=74820&toCityId=194483&fromCityName=Kolkata&toCityName=Suri&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/motihari-to-agra?fromCityId=80302&toCityId=1290&fromCityName=Motihari&toCityName=Agra&busType=Any&opId=17889&onward=18-Jul-2024',
        'https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh?fromCityId=733&toCityId=95048&fromCityName=Delhi&toCityName=Hamirpur%20(Himachal%20Pradesh)&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/badaun-to-delhi?fromCityId=94833&toCityId=733&fromCityName=Badaun&toCityName=Delhi&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/kadapa-to-bangalore?fromCityId=284&toCityId=122&fromCityName=Kadapa&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/chennai-to-bangalore?fromCityName=Chennai&fromCityId=123&toCityName=Bangalore&toCityId=122&busType=Any&opId=17101&onward=18-Jul-2024'
        'https://www.redbus.in/bus-tickets/chennai-to-hyderabad?fromCityName=Chennai&fromCityId=123&toCityName=Hyderabad&toCityId=124&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad?fromCityId=124&toCityId=70995&fromCityName=Hyderabad&toCityName=Adilabad&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/kolkata-to-haldia?fromCityId=74820&toCityId=193533&fromCityName=Kolkata&toCityName=Haldia&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/delhi-to-sitapur?fromCityId=733&toCityId=1438&fromCityName=Delhi&toCityName=Sitapur%20(Uttar%20Pradesh)&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/ongole-to-hyderabad?fromCityId=135&toCityId=124&fromCityName=Ongole&toCityName=Hyderabad&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/delhi-to-shimla?fromCityId=733&toCityId=1285&fromCityName=Delhi&toCityName=Shimla&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur?fromCityId=470&toCityId=1169&fromCityName=Udaipur&toCityName=Jodhpur&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha?fromCityId=201668&toCityId=74706&fromCityName=Barasat%20(West%20Bengal)&toCityName=Digha&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND'
        'https://www.redbus.in/bus-tickets/ludhiana-to-delhi-airport?fromCityId=736&toCityId=94113&fromCityName=Ludhiana&toCityName=Delhi%20Airport&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        'https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal?fromCityId=74706&toCityId=201668&fromCityName=Digha&toCityName=Barasat%20(West%20Bengal)&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND',
        ]

all_bus_data = []

# Iterate through the route links
for route_link in route_links:
    bus_data = scrape_route(route_link)
    if bus_data:
        all_bus_data.extend(bus_data)
    else:
        print(f"Could not scrape the data for: {route_link}")

    # Adding a sleep time to prevent too many rapid requests
    time.sleep(20)

# Save all the data to a single CSV file
if all_bus_data:
    df = pd.DataFrame(all_bus_data)
    csv_file_path = os.path.join(home_dir, f"bus_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
    df.to_csv(csv_file_path, index=False)
    print(f"All scraped data saved to: {csv_file_path}")
else:
    print("No data was scraped.")

https://www.redbus.in/bus-tickets/kozhikode-to-bangalore?fromCityId=74661&toCityId=122&fromCityName=Kozhikode&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND
Route Name: Kozhikode to Bangalore Bus
https://www.redbus.in/bus-tickets/kozhikode-to-bangalore?fromCityId=74661&toCityId=122&fromCityName=Kozhikode&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND
KSRTC (Kerala) - 788
False
Super Fast Non AC Seater (2+3)
07:00
09h 00m
16:00
4.5
INR 432
KSRTC (Kerala) - 2132
False
Swift Deluxe Non AC Air Bus (2+2)
07:32
08h 57m
16:29
3.3
INR 567
KSRTC (Kerala) - 580
False
Super Deluxe Non AC Seater Air Bus (2+2)
08:06
08h 30m
16:36
3.4
INR 469
KSRTC (Kerala) - 53
False
SWIFT-GARUDA A/C SEATER BUS
08:29
07h 51m
16:20
3.6
INR 627
KSRTC (Kerala) - 579
False
Super Deluxe Non AC Seater Air Bus (2+2)
10:00
08h 30m
18:30
2.9
INR 469
KSRTC (Kerala) - 55
False
SWIFT-GARUDA A/C SEATER BUS
12:02
07h 53m
19:55
3.8
INR 627
KSRTC

# Connecting to MySQL and Creating the Table 

In [ ]:
# Connect to MySQL
con = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password"
)

cursor = con.cursor()

# Create database
cursor.execute("CREATE DATABASE IF NOT EXISTS redbus")
cursor.execute("USE redbus")

# Create table
create_table_query = """
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        route_name TEXT,
        route_link TEXT,
        busname TEXT,
        bustype TEXT,
        departing_time DATETIME,
        duration TEXT,
        reaching_time DATETIME,
        star_rating FLOAT,
        price DECIMAL(10, 2),
        seats_available INT
    );
"""
cursor.execute(create_table_query)

# Reading the CSV file

In [3]:
# Read the CSV file into a DataFrame
csv_file_path = r"C:\Users\suffu\bus_data.csv"
df = pd.read_csv(csv_file_path)

In [4]:
df

,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Kozhikode to Bangalore Bus,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 578,Super Deluxe Non AC Seater Air Bus (2+2),18:00,10h 00m,4:00,3.1,INR 513,26
1,Kozhikode to Bangalore Bus,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 72,SWIFT-GARUDA A/C SEATER BUS,19:00,08h 35m,3:35,3.4,INR 688,10
2,Kozhikode to Bangalore Bus,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1583,Super Deluxe Non AC Seater Air Bus (2+2),20:02,09h 19m,5:21,3.7,INR 640,18
3,Kozhikode to Bangalore Bus,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 234,Swift Deluxe Non AC Air Bus (2+2),21:00,09h 00m,6:00,4.0,INR 620,15
4,Kozhikode to Bangalore Bus,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 1584,Super Deluxe Non AC Seater Air Bus (2+2),21:29,09h 09m,6:38,3.8,INR 640,28
...,...,...,...,...,...,...,...,...,...,...
469,Digha to Barasat (West Bengal) Bus,https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,4:30,1.0,570,60
470,Badaun to Delhi Bus,https://www.redbus.in/bus-tickets/badaun-to-de...,UPSRTC - BDN0039,Ordinary Non AC Seater 2+3,6:00,06h 31m,12:31,3.3,INR 411,20
471,Badaun to Delhi Bus,https://www.redbus.in/bus-tickets/badaun-to-de...,UPSRTC - GRH0091,Ordinary Non AC Seater 2+3,6:10,07h 00m,13:10,3.3,INR 403,20
472,Badaun to Delhi Bus,https://www.redbus.in/bus-tickets/badaun-to-de...,UPSRTC - GRH0090,Ordinary Non AC Seater 2+3,05:50\t,07h 00m\t,12:50\t,3.3,INR 403,20


# Converting the departing, reaching time to datetime format and removing Non- Numeric Character in Price

In [ ]:
# Function to convert time to datetime format
def convert_to_datetime(time_str):
    try:
        return datetime.strptime(time_str, '%H:%M').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        return None

# Function to clean price values
def clean_price(price_str):
    # Remove any non-numeric characters except the decimal point
    price_str = re.sub(r'[^\d.]', '', price_str)
    try:
        return float(price_str)
    except ValueError:
        return None

# Convert time columns to proper datetime format
df['departing_time'] = df['departing_time'].apply(convert_to_datetime)
df['reaching_time'] = df['reaching_time'].apply(convert_to_datetime)

# Clean price values
df['price'] = df['price'].apply(clean_price)

# Inserting the data into MySQL Table

In [53]:

# Insert data into the MySQL table
for _, bus in df.iterrows():
    cursor.execute("""
        INSERT INTO bus_routes (
            route_name, route_link, busname, bustype, departing_time,
            duration, reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        bus['route_name'], bus['route_link'], bus['busname'], bus['bustype'], bus['departing_time'],
        bus['duration'], bus['reaching_time'], bus['star_rating'], bus['price'], bus['seats_available']
    ))

# Commit the transaction
con.commit()

# Close the cursor and connection
cursor.close()
con.close()

print("Data inserted successfully.")

Data inserted successfully.


# Displaying the data from MySQL

In [54]:
import mysql.connector

# Connect to MySQL
con = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password"
    database= "redbus"
)
cursor = con.cursor()

# Execute SELECT query
cursor.execute("SELECT * FROM bus_routes")

# Fetch all rows
rows = cursor.fetchall()

# Display the data
print("ID | route_name | route_link | busname | bustype | departing_time | duration | reaching_time | star_rating | price | seats_available")
print("-" * 100)  # Separator line

for row in rows:
    print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]} | {row[4]} | {row[5]} | {row[6]} | {row[7]} | {row[8]} | {row[9]} | {row[10]}")

# Close cursor and connection
cursor.close()
con.close()

ID | route_name | route_link | busname | bustype | departing_time | duration | reaching_time | star_rating | price | seats_available
----------------------------------------------------------------------------------------------------
1 | Kozhikode to Bangalore Bus | https://www.redbus.in/bus-tickets/kozhikode-to-bangalore?fromCityId=74661&toCityId=122&fromCityName=Kozhikode&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND | KSRTC (Kerala) - 578 | Super Deluxe Non AC Seater Air Bus (2+2) | 1900-01-01 18:00:00 | 10h 00m | 1900-01-01 04:00:00 | 3.1 | 513.00 | 26
2 | Kozhikode to Bangalore Bus | https://www.redbus.in/bus-tickets/kozhikode-to-bangalore?fromCityId=74661&toCityId=122&fromCityName=Kozhikode&toCityName=Bangalore&busType=Any&opId=0&onward=18-Jul-2024&srcCountry=IND&destCountry=IND | KSRTC (Kerala) - 72 | SWIFT-GARUDA A/C SEATER BUS | 1900-01-01 19:00:00 | 08h 35m | 1900-01-01 03:35:00 | 3.4 | 688.00 | 10
3 | Kozhikode to Bangalore Bus | h

# Connecting the data from the data base

In [8]:
def get_data_from_db():
    # Connect to the database
    con = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password"
    database="redbus"

    )
        
    query = "SELECT * FROM bus_routes"
    df = pd.read_sql(query, connection)
    connection.close()
    return df

# Cleaning data 

In [9]:
# Function to clean data (fill NaN values and convert types)
def clean_data(df):
    df['star_rating'] = df['star_rating'].fillna(0)
    df['price'] = df['price'].fillna(0)
    df['seats_available'] = df['seats_available'].fillna(df['seats_available'].median()).astype(int)
    return df

# Streamlit Application

In [10]:
# Streamlit application
def main():
    st.title('Bus Route Information')

    # Get data from database
    df = get_data_from_db()

    # Clean data
    df = clean_data(df)

    # Sidebar filters
    st.sidebar.header('Filters')
    route_filter = st.sidebar.multiselect('Select Route', df['route_name'].unique())
    bus_type_filter = st.sidebar.multiselect('Select Bus Type', df['bustype'].unique())
    star_rating_filter = st.sidebar.slider('Select Star Rating', min_value=float(df['star_rating'].min()), max_value=float(df['star_rating'].max()), step=0.1)
    price_filter = st.sidebar.slider('Select Price Range', min_value=float(df['price'].min()), max_value=float(df['price'].max()), step=1.0)
    seats_available = st.sidebar.number_input("Seats Available", min_value=0, step=1)

    # Filter data based on user inputs
    filtered_data = df
    if route_filter:
        filtered_data = filtered_data[filtered_data['route_name'].isin(route_filter)]
    if bus_type_filter:
        filtered_data = filtered_data[filtered_data['bustype'].isin(bus_type_filter)]
    filtered_data = filtered_data[filtered_data['star_rating'] >= star_rating_filter]
    filtered_data = filtered_data[filtered_data['price'] <= price_filter]

    # Display filtered data
    st.write(filtered_data)

if __name__ == "__main__":
    main()

C:\Users\suffu\AppData\Local\Temp\ipykernel_13600\1608816730.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
2024-07-18 13:01:58.103 Session state does not function when running a script without `streamlit run`
2024-07-18 13:01:58.175 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('1900-01-01 21:45:00') with type Timestamp: tried to convert to int64", 'Conversion failed for column departing_time with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


# Streamlit Application: Database Connection, Data Cleaning, and Visualization
This heading captures the key aspects of your project, focusing on connecting to the database, cleaning the data, and visualizing it using Streamlit in VS Code.